# Fetch Data through API

In [6]:
import requests
import json
import csv
import pandas as pd
from requests.api import head

url = 'https://data.ny.gov/api/views/4a2x-yp8g/rows.json?accessType=DOWNLOAD'
headers = { 
    'Accept': 'application/json', 
    'Content-Type': 'application/json'
}

#API call to get records
response = requests.request("GET", url, headers=headers,data={})
decoded_data = response.text.encode().decode('utf-8-sig')
data = json.loads(decoded_data)

#API result
columns = data['meta']['view']['columns']
rows = data['data']
excluded_columns = 8

# Convert JSON data to CSV format

In [ ]:
# Save to a csv file
with open('Output.csv', 'w', newline='', encoding='utf-8') as outputFile:
    fieldnames = [column['name'] for column in columns][excluded_columns:27] # Extract column names from metadata
    writer = csv.writer(outputFile)
    writer.writerow(fieldnames)
    # Write rows to the CSV file
    for row in rows:
        writer.writerow(row[excluded_columns:27])   
        

# Replace null values with NA

In [9]:
csv_data = pd.read_csv('Output.csv')
csv_data.fillna('NA', inplace=True)
csv_data.to_csv('CleanedOutput.csv', index=False)

Connecting DB in the Debian server through MySQL

In [ ]:
%pip install mysql-connector-python
%pip install pymysql
%pip install pandas

In [1]:
import mysql.connector as msql
from mysql.connector import Error
import csv

try:
    conn = msql.connect(
       host="localhost", 
       database="x22105522DB", 
       user="root", 
       password="Karthika@31"
    )
    if conn.is_connected():
      cursor = conn.cursor()
      cursor.execute("select database();")
      record = cursor.fetchone()

      print("You're connected to database: ", record)
      cursor.execute('DROP TABLE IF EXISTS EnergyEfficiencyProject;')

      print('Creating table....')
      cursor.execute("CREATE TABLE EnergyEfficiencyProject (Reporting_Period CHAR(30) NOT NULL, Project_ID CHAR(30) NOT NULL, Project_County CHAR(30) NOT NULL, Project_City CHAR(30) NOT NULL, Project_ZIP CHAR(30) NOT NULL, Gas_Utility CHAR(30) NOT NULL, Electric_Utility CHAR(30) NOT NULL, Project_Completion_Date CHAR(30) NOT NULL, Total_Project_Cost CHAR(30) NOT NULL, Heating_FuelType CHAR(30) NOT NULL, Year_Home_Built CHAR(250) NOT NULL, Size_Of_Home CHAR(30) NOT NULL, Number_Of_Units CHAR(30) NOT NULL, Job_Type CHAR(30) NOT NULL, Type_Of_Dwelling CHAR(30) NOT NULL, Measure_Type CHAR(30) NOT NULL, Annual_Savings_kWh CHAR(30) NOT NULL, Annual_Savings_MMBtu CHAR(30) NOT NULL, Energy_Savings CHAR(30) NOT NULL)")
      print("EnergyEfficiencyProject table is created....")

except Error as e:
    print("Error while connecting to MySQL", e)

# INSERT DATA INTO THE TABLE EnergyEfficiencyProject

# Open the CSV file and read its contents
with open('CleanedOutput.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader) # skip the header row
    for row in reader:
        
        #Insert each row into the database
        cursor = conn.cursor()
        cursor.execute("INSERT INTO EnergyEfficiencyProject (Reporting_Period, Project_ID, Project_County, Project_City, Project_ZIP, Gas_Utility, Electric_Utility, Project_Completion_Date, Total_Project_Cost, Heating_FuelType, Year_Home_Built, Size_Of_Home, Number_Of_Units, Job_Type, Type_Of_Dwelling, Measure_Type, Annual_Savings_kWh, Annual_Savings_MMBtu, Energy_Savings) VALUES (%s, %s, %s,%s, %s, %s,%s, %s, %s,%s, %s, %s,%s, %s, %s,%s, %s, %s,%s);", (row[0], row[1], row[2],row[3],row[4],row[5],row[6],row[7],row[8],row[9],row[10],row[11],row[12],row[13],row[14],row[15],row[16],row[17],row[18]))
        conn.commit()
        print("Table inserted...")

# Close the database connection
conn.close()

You're connected to database:  ('x22105522db',)
Creating table....
EnergyEfficiencyProjectsss table is created....


# Creating DB through PostGreSQL

In [ ]:
%pip install psycopg2

DB connection

In [ ]:

import psycopg2

try:
    dbConnection = psycopg2.connect(user = "dap", password = "dap", host = "87.44.4.70", port = "5432", database = "postgres")

    dbConnection.set_isolation_level(0) # AUTOCOMMIT
    dbCursor = dbConnection.cursor()

    dbCursor.execute('CREATE DATABASE postgres;')
    dbCursor.close()

except (Exception, psycopg2.Error) as dbError :
    print ("Error while connecting to PostgreSQL", dbError)


finally:
    if(dbConnection): dbConnection.close()


Creating tables

In [ ]:

dbConnection = psycopg2.connect(user = "dap", password = "dap", host = "87.44.4.70", port = "5432", database = "postgres")
dbCursor = dbConnection.cursor()
dbCursor.execute("CREATE TABLE EnergyEfficiencyProjects (Reporting_Period VARCHAR(50) NOT NULL, Project_ID VARCHAR(50) NOT NULL, Project_County VARCHAR(50) NOT NULL, Project_City VARCHAR(50) NOT NULL, Project_ZIP VARCHAR(50) NOT NULL, Gas_Utility VARCHAR(50) NOT NULL, Electric_Utility VARCHAR(50) NOT NULL, Project_Completion_Date VARCHAR(50) NOT NULL, Total_Project_Cost VARCHAR(50) NOT NULL, Heating_FuelType VARCHAR(50) NOT NULL, Year_Home_Built CHAR(250) NOT NULL, Size_Of_Home VARCHAR(50) NOT NULL, Number_Of_Units VARCHAR(50) NOT NULL, Job_Type VARCHAR(50) NOT NULL, Type_Of_Dwelling VARCHAR(50) NOT NULL, Measure_Type VARCHAR(50) NOT NULL, Annual_Savings_kWh VARCHAR(50) NOT NULL, Annual_Savings_MMBtu VARCHAR(50) NOT NULL, Energy_Savings VARCHAR(50) NOT NULL)")

# Commit the transaction and close the cursor and connection
dbConnection.commit()
dbCursor.close()
dbConnection.close()


Inserting data in table

In [ ]:
dbConnection = psycopg2.connect(user = "dap", password = "dap", host = "87.44.4.70", port = "5432", database = "postgres")
dbCursor = dbConnection.cursor()

insert_statement = "INSERT INTO EnergyEfficiencyProjects (Reporting_Period, Project_ID, Project_County, Project_City, Project_ZIP, Gas_Utility, Electric_Utility, Project_Completion_Date, Total_Project_Cost, Heating_FuelType, Year_Home_Built, Size_Of_Home, Number_Of_Units, Job_Type, Type_Of_Dwelling, Measure_Type, Annual_Savings_kWh, Annual_Savings_MMBtu, Energy_Savings) VALUES (%s, %s, %s,%s, %s, %s,%s, %s, %s,%s, %s, %s,%s, %s, %s,%s, %s, %s,%s)"

# Open the CSV file
with open('CleanedOutput.csv', 'r') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    next(csv_reader)
    
    for row in csv_reader:
        data_insert = (row[0], row[1], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[9], row[10], row[11], row[12], row[13], row[14], row[15], row[16], row[17], row[18])
        # Executing the INSERT statement
        dbCursor.execute(insert_statement, data_insert)


# Commit the transaction and close the cursor and connection
dbConnection.commit()
dbCursor.close()
dbConnection.close()